### In this file, we will generate our E0 feature data

Instructions: 

You should start by writing a notebook to create the dataset. Note that if you use many molecules, using MOPAC could take a while. If you decide to optimize the geometry of all the molecules in the Solubility datasets, ot will take at least 12 hours. Make sure to save the result so that you don't have to compute it multiple times!

In [5]:
# Pandas dataframe, apply function to whole dataframe
# Make sure it works on a small dataset. Save the results
# Create dataset associated smiles to energy. Then, run overnight on all data
# Once dataset ready, can move on to next step

import pandas as pd
import os

In [6]:
# Combine Datasets
def generate_combined_dataset(dir_path, fnames, out_name="", debug=False):
    combined = None
    print("Outname is", out_name)
    for f in fnames:
        df = pd.read_csv(dir_path + f)
        if combined is None:
            combined = df.copy()
        else:
            ## Here we do not check for duplicates as Professor Tristan in class informed me that there should not be any duplicate data between the sets
            combined = pd.concat([combined, df])
        if debug:
            print("Added " + str(f) + " with dim", df.shape)
    if debug:
        print("Our combined dataset has", combined.shape[0], "rows with", combined.shape[1], "features.")
    
    if out_name != "":
        if out_name.endswith(".csv"):
            if os.path.exists(out_name):
                print("Combined file already exists at specified output path... rename or replace it first.")
            else:
                combined.to_csv(out_name, index=False)
        else:
            print("ERROR: Output filename must end in .csv")
    return combined
    
# Restriction: "It is forbidden to use any dataset other than A, B, C, D, F, G, H, I"
# So, we make sure we only use these datasets in our data preparation

generate_combined_dataset("../../Data/Solubility/",
                          ["dataset-H.csv", "dataset-I.csv"],
                          "train_data_preview.csv")

generate_combined_dataset("../../Data/Solubility/",
                          ["dataset-A.csv", "dataset-B.csv", "dataset-C.csv",
                          "dataset-D.csv", "dataset-F.csv", "dataset-G.csv",
                          "dataset-H.csv", "dataset-I.csv"], "train_data.csv")

Outname is train_data_preview.csv


FileNotFoundError: [Errno 2] No such file or directory: '../../Data/Solubility/dataset-H.csv'

In [8]:
from rdkit import Chem
from rdkit.Chem import AllChem
from ase import Atoms
from ase.io import read
from ase.calculators.mopac import MOPAC

from tqdm.notebook import tqdm_notebook

# If necessary, can add a batch parameter to start at a given index and to save batches of data at intervals
def calc_potential(from_df):
    
    def get_mopac(smile):
        try:
            # Create the rdkit molecule object in Cartesian space
            mol = Chem.AddHs(Chem.MolFromSmiles(smile))
            AllChem.EmbedMolecule(mol)

            # Save and reload the information into a mol object from ase (instead of rdkit)
            Chem.rdmolfiles.MolToXYZFile(mol, 'init.xyz')
            mol = read('init.xyz')

            # With this ase object, add quantum mechanic calculations
            mol.calc = MOPAC(label='TMP', task='UHF BONDS GRADS')
            return mol.get_potential_energy()
        except:
            pass
        
    tqdm_notebook.pandas(desc="Quantum Calculations for Minimum Potential Energy (Molecule Number)")
    
    # Would in theory progress_apply each batch if we needed
    
    from_df['Min_PE'] = from_df['SMILES'].progress_apply(get_mopac)
    return from_df
    


# CAUTION TEAM ELECTRON
You only need to run your own code block here. I chunked it out so each of us have about an even amount of data to churn through so we can do this at 3x speed. Be advised it may take ~30 hours to go through all of this data, so for this reason I have it save to the csv file every time you complete one of the data sets. That's why there are multiple lines of calculating and saving

In [11]:
data_path = "../MLQC_HW/Data/Solubility/"

In [5]:
### RJ run this ###

# Will do C, D, F (I already did H and I while testing)

# C
new_df = calc_potential(pd.read_csv(data_path + "dataset-C.csv"))

# Only keep our SMILES and Min_PE
new_df[["SMILES", "Min_PE"]].to_csv('Data/pe_data_C.csv', index=False)


/opt/conda/lib/python3.8/site-packages/tqdm/std.py:702: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


Quantum Calculations for Minimum Potential Energy (Molecule Number):   0%|          | 0/2603 [00:00<?, ?it/s]

In [ ]:
### RJ run this ###
# D
new_df = calc_potential(pd.read_csv(data_path + "dataset-D.csv"))
new_df[["SMILES", "Min_PE"]].to_csv('Data/pe_data_D.csv', index=False)

/opt/conda/lib/python3.8/site-packages/tqdm/std.py:702: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


Quantum Calculations for Minimum Potential Energy (Molecule Number):   0%|          | 0/2115 [00:00<?, ?it/s]

In [ ]:
# F
new_df = calc_potential(pd.read_csv(data_path + "dataset-F.csv"))
new_df[["SMILES", "Min_PE"]].to_csv('Data/pe_data_F.csv', index=False)

/opt/conda/lib/python3.8/site-packages/tqdm/std.py:702: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


Quantum Calculations for Minimum Potential Energy (Molecule Number):   0%|          | 0/1210 [00:00<?, ?it/s]

In [ ]:
### RONAN RUN THIS ###

# This will do the computations on dataset A

# A
new_df = calc_potential(pd.read_csv(data_path + "dataset-A.csv"))
new_df[["SMILES", "Min_PE"]].to_csv('Data/pe_data_A.csv', index=False)

In [ ]:
### JOSHUA RUN THIS ###

# This will do the computations on dataset B, G

# B
new_df = calc_potential(pd.read_csv(data_path + "dataset-B.csv"))
new_df[["SMILES", "Min_PE"]].to_csv('Data/pe_data_B.csv', index=False)

In [ ]:
### JOSHUA RUN THIS ###
# G
new_df = calc_potential(pd.read_csv(data_path + "dataset-B.csv"))
new_df[["SMILES", "Min_PE"]].to_csv('Data/pe_data_G.csv', index=False)